In [ ]:
import plotly.graph_objects as go
import pandas as pd
import yfinance as yf

# Загрузка данных
df = yf.download("BTC-USD", start='2024-03-30', end='2024-06-02', interval='1h').drop(columns=['Adj Close'])
df.index = pd.to_datetime(df.index)
df = df.droplevel("Ticker", axis=1)

[*********************100%***********************]  1 of 1 completed


In [ ]:
df

Price,Close,High,Low,Open,Volume
Datetime,,,,,
2024-03-30 00:00:00+00:00,69960.109375,69960.109375,69807.460938,69889.773438,0
2024-03-30 01:00:00+00:00,70023.867188,70171.398438,69884.984375,69969.039062,61065216
2024-03-30 02:00:00+00:00,69976.976562,70039.960938,69896.804688,70026.062500,0
2024-03-30 03:00:00+00:00,69802.085938,70056.468750,69802.085938,69979.453125,0
2024-03-30 04:00:00+00:00,70105.617188,70109.734375,69807.390625,69815.187500,0
...,...,...,...,...,...
2024-06-01 19:00:00+00:00,67708.835938,67728.062500,67638.000000,67711.250000,0
2024-06-01 20:00:00+00:00,67734.062500,67734.062500,67659.718750,67695.601562,0
2024-06-01 21:00:00+00:00,67730.257812,67761.507812,67664.460938,67741.742188,0


In [51]:
# Определение уровней
window = 24  # Размер окна для расчета уровней

df['Pivot'] = (df['High'].rolling(window=window).max() +      # Расчет Пивот
               df['Low'].rolling(window=window).min() +
               df['Close'].rolling(window=window).mean()) / 3

df['Resistance'] = 2 * df['Pivot'] - df['Low'].rolling(window=window).min() # Расчет уровня сопротивления 
df['Support'] = 2 * df['Pivot'] - df['High'].rolling(window=window).max() # Расчет уровня поддержки



# # Списки для хранения данных о пробоях
breakthroughs_support = []
breakthroughs_resistance = []

# Проверка пробития уровней
for i in range(window + 1, len(df)):
    current_high = df['High'].iloc[i]
    current_low = df['Low'].iloc[i]
    prev_support = df['Support'].iloc[i-1]
    prev_resistance = df['Resistance'].iloc[i-1]
    
    # Проверяем, пересекла ли цена уровень поддержки снизу вверх
    if df['Low'].iloc[i-1] > prev_support and current_low <= prev_support:
        breakthroughs_support.append((df.index[i], current_low))
    
    # Проверяем, пересекла ли цена уровень сопротивления сверху вниз
    if df['High'].iloc[i-1] < prev_resistance and current_high >= prev_resistance:
        breakthroughs_resistance.append((df.index[i], current_high))

In [38]:
# Создание графика
fig = go.Figure()

# Добавление линии цены
fig.add_trace(go.Scatter(x=df.index, y=df['Close'], mode='lines', name='Цена'))

# Добавление уровней поддержки и сопротивления
fig.add_trace(go.Scatter(x=df.index, y=df['Support'], mode='lines', name='Поддержка',
                         line=dict(color='green', width=1, dash='dash')))
fig.add_trace(go.Scatter(x=df.index, y=df['Resistance'], mode='lines', name='Сопротивление',
                         line=dict(color='red', width=1, dash='dash')))
fig.add_trace(go.Scatter(x=df.index, y=df['Pivot'], mode='lines', name='Пивот',
                         line=dict(color='black', width=1, dash='dash')))

# Добавление точек пробития
if breakthroughs_support:
    breakthroughs_support_df = pd.DataFrame(breakthroughs_support, columns=['Date', 'Price'])
    fig.add_trace(go.Scatter(x=breakthroughs_support_df['Date'], y=breakthroughs_support_df['Price'],
                             mode='markers', name='Пробитие поддержки',
                             marker=dict(color='green', size=10)))

if breakthroughs_resistance:
    breakthroughs_resistance_df = pd.DataFrame(breakthroughs_resistance, columns=['Date', 'Price'])
    fig.add_trace(go.Scatter(x=breakthroughs_resistance_df['Date'], y=breakthroughs_resistance_df['Price'],
                             mode='markers', name='Пробитие сопротивления',
                             marker=dict(color='red', size=10)))

# Настройка оформления графика
fig.update_layout(title='График цены с уровнями поддержки и сопротивления и пробитиями',
                  xaxis_title='Дата',
                  yaxis_title='Цена',
                  xaxis_rangeslider_visible=False)

# Отображение графика
fig.show()